In [1]:
import data
import os, glob
import matplotlib.pyplot as plt
import cv2
import numpy as np

from networks import CNNFeatureLSTM

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [14]:
d = data.ClogData('/ssd_icybox2_1TB/jason/clog-loss-data/', size='micro')

In [4]:
d.crop_videos()

CROPPING ALL VIDEOS TO SIZE  (224, 224) ...
100%|██████████| 4798/4798 [00:00<00:00, 100503.26it/s]


In [5]:
import tensorflow as tf

In [6]:
cnn = tf.keras.applications.VGG16()

In [7]:
model = CNNFeatureLSTM(cnn)

In [8]:
model.summary()

CNN Model: vgg16
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 128)               545280    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 553,601
Trainable params: 553,601
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
model.fit(d, epochs=50, callbacks=[EarlyStopping(monitor='val_matthewcorr', 
                                                 patience=3,
                                                 mode='max',
                                                 restore_best_weights=True)])

LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
Train on 1919 samples, validate on 480 samples
Epoch 1/50
1919/1919 [==============================] - 8s 4ms/sample - loss: 0.6070 - accuracy: 0.6972 - matthewcorr: nan - val_loss: 0.5799 - val_accuracy: 0.7167 - val_matthewcorr: nan
Epoch 2/50
1919/1919 [==============================] - 3s 2ms/sample - loss: 0.5500 - accuracy: 0.7264 - matthewcorr: nan - val_loss: 0.5837 - val_accuracy: 0.7188 - val_matthewcorr: nan
Epoch 3/50
1919/1919 [==============================] - 3s 2ms/sample - loss: 0.5145 - accuracy: 0.7410 - matthewcorr: nan - val_loss: 0.5939 - val_accuracy: 0.7146 - val_matthewcorr: 0.2003
Epoch 4/50
1919/1919 [==============================] - 3s 2ms/sample - loss: 0.4866 - accuracy: 0.7718 - matthewcorr: nan - val_loss: 0.5811 - val_accuracy: 0.7333 - val_matthewcorr: 0.2663
Epoch 5/50
1919/1919 [==============================] - 3s 2ms/sample - loss: 0.4464 - accuracy: 0.7952 - matthewcorr: 0.4574 - val_loss:

In [11]:
predictions = model.predict(d, train=False)

  0%|          | 54/14160 [05:50<25:27:58,  6.50s/it] 


KeyboardInterrupt: 

In [12]:
from joblib import Parallel

Parallel(n_jobs=10)(d.load(i, train=False, ret=False) for i in range(len(d.test)))

KeyboardInterrupt: 